# SGD

Este notebook contem a implementação da regressão linear via gradiente descendente estocástico, desenvolvida para o curso CK0149 - GARIMPAGEM DE DADOS 2016.2

## 1. Handler de dados

Inicialmente devemos carregar os dados do dataset em uma matriz. Para isto, usaremos o NumPy.

In [13]:
import numpy as np
import math
from random import seed
from random import randrange
import warnings
warnings.filterwarnings("ignore")

seed()

data = np.loadtxt("bike_sharing.csv", delimiter = ',')
len(data)

17379

Agora, precisamos normalizar os dados e separá-los em k-folds

In [2]:
def findMinMax(dataset):
    output = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        least = min(col_values)
        maximum = max(col_values)
        output.append([least, maximum])
    return output

def normalize(dataset, minmax):
    for row in dataset:
        for i in range(len(row) - 1):
            if not (minmax[i][0]==0 and minmax[i][1]==1):
                row[i] = (row[i] - minmax[i][0])/(minmax[i][1] - minmax[i][0])
            
def kFoldSplit(dataset, k):
    splitData = []
    copy = list(dataset)
    fold_size = len(dataset) / k
    for i in range(k):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(copy))
            fold.append(copy.pop(index))
        splitData.append(fold)
    return splitData

## 2. Fazendo predições

Ao completar a separação dos dados de treino e teste, devemos então iniciar a implementação do $SGD$. Para isso, precisamos inicialmente de um método para predizer a classe de um exemplo com base em no conjunto de coeficientes do nosso modelo. Dessa forma, criamos a função predict:

In [3]:
def predict(example, coefficients):
    y = coefficients[0]
    
    for i in range(len(example) - 1):
        y += coefficients[i+1]*example[i]
    
    return y

## 3. Estimando os coeficientes do modelo

Nosso modelo de predição é descrito em função de um conjunto de coeficientes, os quais são ajustados iterativamente durante a fase de treinamento. A função definida abaixo atualiza o valor dos coeficientes durante as épocas de treinamento:

In [4]:
def updateCoefficients(trainingSet, alpha, epochs):
    coef = [0.0 for i in range(len(trainingSet[0]))]
    for epoch in range(epochs):
        for row in trainingSet:
            y = predict(row, coef)
            
            error = y - row[-1]
            
            coef[0] = coef[0] - alpha * error
            
            for i in range(len(row) - 1):
                coef[i+1] = coef[i + 1] - alpha * error * row[i]
    return coef        

## 4. Regressão linear

Agora que temos as funcoes de predição e de atualização dos coeficientes, podemos implementar a funcao principal da regressão linear.

In [5]:
def linearRegression(train, test, alpha, epochs):
    predictions = list()
    output = {}
    coef = updateCoefficients(train, alpha, epochs)
    for row in test:
        y = predict(row, coef)
        predictions.append(y)
    output.update({"model": coef, "predictions": predictions})
    return output

## 5. Avaliando os resultados

Agora precisamos de alguma métrica de avaliação do resultado obtido pelo algoritmo. Para isso, implementaremos uma função que calcula a raíz do erro quadrático médio e uma que avalia o algoritmo em si, utilizando os K Folds implementados acima.

In [6]:
def meanQuadraticError(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        prediction_error = actual[i] - predicted[i]
        sum_error += (prediction_error ** 2)
    mean = sum_error/float(len(actual))
    
    return math.sqrt(mean)

In [7]:
def evaluate(dataset, algorithm, nFolds, *args):
    folds = kFoldSplit(dataset, nFolds)
    foldPos = -1
    output = {}
    output.update({'folds': folds})
    output.update({'actual': []})
    output.update({'predicted': []})
    output.update({'scores': []})
    for fold in folds:
        foldPos += 1
        train_set = list(folds)
        train_set.pop(foldPos)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        result = algorithm(train_set, test_set, *args)
        predicted = result['predictions']
        output.update({'model': result['model']})
        actual = [row[-1] for row in fold]
        rmse = meanQuadraticError(actual, predicted)
        output['scores'].append(rmse)
        output['predicted'].append(predicted)
        output['actual'].append(actual)
    
    return output

## 6. Executando o algoritmo

Agora devemos criar uma função main que chama as funções acima e calcula o resultado.

In [8]:
minmax = findMinMax(data)
normalize(data, minmax)

k = 5
alpha = 1e-3
epochs = 65
output = evaluate(data, linearRegression, k, alpha, epochs)
actual = output['actual']
predicted = output['predicted']
scores = output['scores']
model = output['model']
print('Scores: %s' % scores)


Scores: [8.024228329944282, 9.281614982929225, 6.599785374096325, 10.872708884767993, 8.674604719713221]


## 7. Métrica R²

A métrica R² foi vista em classe e é uma boa maneira de avaliar o resultado. Abaixo, usando a implementação do scikit-learn dessa métrica, foi calculada a pontuação obtida ao usar cada fold como conjunto de teste.

In [9]:
from sklearn import metrics
for i in range(len(actual)):
    print metrics.r2_score(actual[i], predicted[i])

0.998040888973
0.997330342553
0.998702889885
0.996407108131
0.997696567792


Dessa forma, podemos ver que os resultados preditos estão bem próximos dos resultados reais.

## 8. Questões propostas

### a) Como você escolheu os parâmetros "número de iterações" e "taxa de aprendizado"? 

O número de iterações consiste do número de repetições (conhecidas como épocas de treinamento) que faremos do calculo dos coeficientes do modelo durante o processo de treinamento. Sabemos que o erro quadrático médio deve cair significativamente nas primeiras épocas de treinamento para então passar a ser praticamente constante. Nesse caso, devemos escolher um número de épocas que seja suficientemente grande para ultrapassar essa fase de queda do erro, uma vez que números de iterações maiores terão erros próximos. Dessa forma, após alguns testes, notou-se que 65 era um bom número.

Já a taxa de aprendizado consiste do tamanho do passo que fazemos na direção do gradiente para se aproximar do mínimo da função de $erro$ $\times$ $coeficientes$. Assim, se escolhermos taxas muito altas, podemos ultrapassar o ponto que queremos e nos afastar do mínimo, porém taxas baixas requerem mais épocas de treinamento, uma vez que a cada iteração se aproximam muito pouco do mínimo. Portanto, uma taxa que se adeque ao conjunto de dados é necessária, de modo que o modelo se atualize da forma desejada. Assim, concluiu-se após testes que 1e-3 era uma boa taxa.

### b) Analise os resultados obtidos. O que o modelo diz sobre a importância de cada atributo?

Abaixo seguem os valores dos coeficientes obtidos para o modelo

In [10]:
model

[0.037142561140993145,
 0.19129688799843453,
 0.58043562326005083,
 -0.040610625404925521,
 1.2828199272820067,
 -0.13366038802548566,
 -0.27695403163458993,
 0.12969200648077039,
 -0.28718810492919189,
 2.2337569406171927,
 0.87767930178240583,
 -1.1318287652372054,
 -0.38991376095354102,
 366.38286262415107,
 882.18020980267966]

Quanto maiores os coeficientes, maior a importância daqueles atributos. Portanto, podemos notar que os dois últimos atributos são significativamente mais importantes para a regressão.

### c) Compare com o resultado obtido pelo metodo dos minimos quadrados (do scikit-learn)

Utilizaremos o linear_model do scikit-learn para efetuar a regressão do mesmo conjunto de dados e então comparar os resultados. Para simplificar, executamos a classificação novamente pelo algoritmo acima, agora com apenas um conjunto de treino e um de teste.

In [11]:
from sklearn import linear_model

rand_data = np.random.permutation(data)

sep = int(0.67*len(rand_data))

train_datask = rand_data[:sep, :-1]
test_datask = rand_data[sep:, :-1]

yTrain_datask = rand_data[:sep, -1]
yTest_datask = rand_data[sep:, -1]

train_data = rand_data[:sep]
test_data = rand_data[sep:]

regr = linear_model.LinearRegression()
regr.fit(train_datask, yTrain_datask)
comparation = linearRegression(train_data, test_data, alpha, epochs)

    

In [14]:
print("Regressao Linear implementada: ", metrics.r2_score(test_data[:,-1], comparation['predictions']))

skpred = []
for d in test_datask:
    skpred.append(regr.predict(d))
    
print("Regressao Linear do scikit-learn: ", metrics.r2_score(rand_data[sep:, -1], skpred))

('Regressao Linear implementada: ', 0.99799195941654528)
('Regressao Linear do scikit-learn: ', 0.99799547525601673)


Como podemos ver, os dois modelos foram incrivelmente proximos.